In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
target_df = pd.read_excel('./data/output/HMDA_2022_CTData.xls')
target_df.head()

,census_tract,TotalOriginations,WhiteNonHispanicOriginations,BlackOriginations,LatinxOriginations,AAPIOriginations,Applications,Denials,Fallout,OriginationRate,DenialRate,FalloutRate
0,55079000101,29,3,10,2,3,52,6,11,0.557692,0.115385,0.211538
1,55079187000,27,16,3,3,1,42,1,7,0.642857,0.023810,0.166667
2,55079018600,15,0,1,11,1,22,1,4,0.681818,0.045455,0.181818
3,55079003500,37,6,21,5,2,61,7,8,0.606557,0.114754,0.131148
4,55079017900,30,23,0,2,1,49,1,11,0.612245,0.020408,0.224490


In [7]:
count = 0
with open('./data/hmda/2022_modified_lar/2022_combined_mlar_header.txt', 'r') as f: 
    for line in f: 
        if count < 100: 
            print(line)
            count += 1
        else: 
            break

activity_year|lei|loan_type|loan_purpose|preapproval|construction_method|occupancy_type|loan_amount|action_taken|state_code|county_code|census_tract|applicant_ethnicity_1|applicant_ethnicity_2|applicant_ethnicity_3|applicant_ethnicity_4|applicant_ethnicity_5|co_applicant_ethnicity_1|co_applicant_ethnicity_2|co_applicant_ethnicity_3|co_applicant_ethnicity_4|co_applicant_ethnicity_5|applicant_ethnicity_observed|co_applicant_ethnicity_observed|applicant_race_1|applicant_race_2|applicant_race_3|applicant_race_4|applicant_race_5|co_applicant_race_1|co_applicant_race_2|co_applicant_race_3|co_applicant_race_4|co_applicant_race_5|applicant_race_observed|co_applicant_race_observed|applicant_sex|co_applicant_sex|applicant_sex_observed|co_applicant_sex_observed|applicant_age|applicant_age_above_62|co_applicant_age|co_applicant_age_above_62|income|purchaser_type|rate_spread|hoepa_status|lien_status|applicant_credit_scoring_model|co_applicant_credit_scoring_model|denial_reason_1|denial_reason_2|den

In [17]:
def split_txt_file(filepath, path_prefix='./data/hmda/', lines_per_file=100000, target_state="WI", target_county="55079"):
    filenames = []
    column_names = []
    root_file_name = filepath.split('/')[-1].split('.txt')[0]
    smallfile = None
    file_counter = 0
    line_counter = 0
    overall_count = 0
    with open(filepath, "r") as bigfile:
        for line in bigfile: 
            if overall_count == 0: 
                column_names = line.strip().split("|")
                overall_count += 1
            columns = line.strip().split("|")
            if len(columns) > 9 and columns[9] == target_state: 
                if line_counter % lines_per_file == 0: 
                    if smallfile: 
                        smallfile.close()
                    small_filename = f"{path_prefix}{root_file_name}_part{file_counter}.txt"
                    smallfile = open(small_filename, "w")
                    file_counter += 1
                    filenames.append(small_filename)
                smallfile.write(line)
                line_counter += 1
                
    if smallfile: 
        smallfile.close()
    return filenames, column_names

In [18]:
filenames, column_names = split_txt_file(
    './data/hmda/2022_modified_lar/2022_combined_mlar_header.txt', 
    path_prefix='./data/hmda/2022_modified_lar/'
)

In [21]:
filenames, column_names

(['./data/hmda/2022_modified_lar/2022_combined_mlar_header_part0.txt',
  './data/hmda/2022_modified_lar/2022_combined_mlar_header_part1.txt',
  './data/hmda/2022_modified_lar/2022_combined_mlar_header_part2.txt'],
 ['activity_year',
  'lei',
  'loan_type',
  'loan_purpose',
  'preapproval',
  'construction_method',
  'occupancy_type',
  'loan_amount',
  'action_taken',
  'state_code',
  'county_code',
  'census_tract',
  'applicant_ethnicity_1',
  'applicant_ethnicity_2',
  'applicant_ethnicity_3',
  'applicant_ethnicity_4',
  'applicant_ethnicity_5',
  'co_applicant_ethnicity_1',
  'co_applicant_ethnicity_2',
  'co_applicant_ethnicity_3',
  'co_applicant_ethnicity_4',
  'co_applicant_ethnicity_5',
  'applicant_ethnicity_observed',
  'co_applicant_ethnicity_observed',
  'applicant_race_1',
  'applicant_race_2',
  'applicant_race_3',
  'applicant_race_4',
  'applicant_race_5',
  'co_applicant_race_1',
  'co_applicant_race_2',
  'co_applicant_race_3',
  'co_applicant_race_4',
  'co_appli

In [27]:
import glob
import os
all_files = glob.glob(os.path.join('./data/hmda/2022_modified_lar', "*.txt"))
all_files = all_files[1:]
all_files

['./data/hmda/2022_modified_lar/2022_combined_mlar_header_part0.txt',
 './data/hmda/2022_modified_lar/2022_combined_mlar_header_part1.txt',
 './data/hmda/2022_modified_lar/2022_combined_mlar_header_part2.txt']

In [72]:
df = pd.concat(
    (pd.read_csv(f, delimiter='|', header=None, names=column_names, low_memory=False) for f in all_files), 
    ignore_index=True
)
# df['census_tract'] = pd.to_numeric(df['census_tract'], errors='coerce').astype("float64")
df['census_tract'] = df['census_tract'].astype("Int64").astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265398 entries, 0 to 265397
Data columns (total 85 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   activity_year                             265398 non-null  int64  
 1   lei                                       265398 non-null  object 
 2   loan_type                                 265398 non-null  int64  
 3   loan_purpose                              265398 non-null  int64  
 4   preapproval                               265398 non-null  int64  
 5   construction_method                       265398 non-null  int64  
 6   occupancy_type                            265398 non-null  int64  
 7   loan_amount                               265398 non-null  int64  
 8   action_taken                              265398 non-null  int64  
 9   state_code                                265398 non-null  object 
 10  county_code         

In [75]:
target_df['census_tract'] = target_df['census_tract'].astype("string")
target_tracts = target_df.census_tract.unique().tolist()
target_tracts = list(map(str, target_tracts))
df_tracts = df.census_tract.unique().tolist()
df_tracts = list(map(str, df_tracts))
missing_tracts = list(set(target_tracts) - set(df_tracts))
assert len(missing_tracts) == 0

In [77]:
# Filter down to the tracts that we care about
mke_df = df[df['census_tract'].notna() & df['census_tract'].isin(target_tracts)]
mke_df.shape

(20352, 85)

In [110]:
target_df.head()

,census_tract,TotalOriginations,WhiteNonHispanicOriginations,BlackOriginations,LatinxOriginations,AAPIOriginations,Applications,Denials,Fallout,OriginationRate,DenialRate,FalloutRate
0,55079000101,29,3,10,2,3,52,6,11,0.557692,0.115385,0.211538
1,55079187000,27,16,3,3,1,42,1,7,0.642857,0.023810,0.166667
2,55079018600,15,0,1,11,1,22,1,4,0.681818,0.045455,0.181818
3,55079003500,37,6,21,5,2,61,7,8,0.606557,0.114754,0.131148
4,55079017900,30,23,0,2,1,49,1,11,0.612245,0.020408,0.224490


In [107]:
test = mke_df.copy()
test.drop_duplicates(inplace=True)
test.query("action_taken == 3 & loan_purpose == 1 & occupancy_type == 1 & census_tract == '55079000101'")

,activity_year,lei,loan_type,loan_purpose,preapproval,construction_method,occupancy_type,loan_amount,action_taken,state_code,...,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose
48688,2022,254900CN1DD55MJDFH69,2,1,2,1,1,195000,3,WI,...,1,1,1,NaN,NaN,NaN,NaN,2,2,2
105570,2022,5493007DIB8YPDM6NR52,1,1,2,1,1,125000,3,WI,...,1,1,6,NaN,NaN,NaN,NaN,2,2,2
172502,2022,549300FGXN1K3HLB1R50,2,1,2,1,1,55000,3,WI,...,1,3,3,NaN,NaN,NaN,NaN,2,2,2
179163,2022,549300AG64NHILB7ZP05,2,1,2,1,1,145000,3,WI,...,1,1,3,NaN,NaN,NaN,NaN,2,2,2
179971,2022,549300SELI3XCH3UZW80,2,1,2,1,1,325000,3,WI,...,1,1,1,NaN,NaN,NaN,NaN,2,2,2
227311,2022,549300MFY0P4D234OB91,2,1,2,1,1,115000,3,WI,...,1,1,1,NaN,NaN,NaN,NaN,2,2,2


In [108]:
# drop duplicates
mke_df.drop_duplicates(inplace=True)

/var/folders/88/q8wq3ss94k3bvmkq1j1f_j0w0000gn/T/ipykernel_55371/966243228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mke_df.drop_duplicates(inplace=True)


In [137]:
mke_purchases_df = mke_df.query("loan_purpose == 1 & occupancy_type == 1")
mke_purchases_df.head()

,activity_year,lei,loan_type,loan_purpose,preapproval,construction_method,occupancy_type,loan_amount,action_taken,state_code,...,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose
163,2022,549300HN58ONH5KNJJ12,2,1,1,1,1,215000,1,WI,...,1,1,1,NaN,NaN,NaN,NaN,2,2,2
278,2022,549300HN58ONH5KNJJ12,1,1,1,1,1,245000,1,WI,...,1,1,2,NaN,NaN,NaN,NaN,2,2,2
398,2022,549300HN58ONH5KNJJ12,1,1,1,1,1,135000,1,WI,...,1,1,2,NaN,NaN,NaN,NaN,2,2,2
406,2022,549300HN58ONH5KNJJ12,1,1,1,1,1,165000,1,WI,...,1,1,6,NaN,NaN,NaN,NaN,2,2,2
505,2022,549300HN58ONH5KNJJ12,1,1,1,1,1,165000,1,WI,...,1,1,1,NaN,NaN,NaN,NaN,2,2,2


In [138]:
## 7656 in test
mke_purchases_df['white_non_hispanic'] = (
    (mke_purchases_df['applicant_ethnicity_1'] == 1) & 
    (mke_purchases_df['applicant_race_1'] == 5)
)


/var/folders/88/q8wq3ss94k3bvmkq1j1f_j0w0000gn/T/ipykernel_55371/3534876697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mke_purchases_df['white_non_hispanic'] = (


In [153]:
tract_summary = (
    mke_purchases_df.groupby("census_tract")
    .apply(lambda g: pd.Series({
        "TotalOriginations": (g["action_taken"] == 1).sum(),
        "WhiteNonHispanicOriginations": ((g["action_taken"] == 1) & (g["applicant_race_1"] == 5) & (g['applicant_ethnicity_1'] == 1)).sum(),
        "BlackOriginations": ((g["action_taken"] == 1) & (g["applicant_race_1"] == 3)).sum(),
        "LatinxOriginations": ((g["action_taken"] == 1) & (g["applicant_ethnicity_1"].isin([1, 11, 12, 13, 14]))).sum(),
        "AAPIOriginations": ((g["action_taken"] == 1) & (g["applicant_race_1"].isin([2, 4, 21, 22, 23, 24, 25, 26, 27]))).sum(),
        "Applications": len(g),
        "Denials": (g["action_taken"] == 3).sum(),
        "Fallout": g["action_taken"].isin([4, 5]).sum(),
        
    }))
    .reset_index()
)
tract_summary.head()

/var/folders/88/q8wq3ss94k3bvmkq1j1f_j0w0000gn/T/ipykernel_55371/4218558576.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,census_tract,TotalOriginations,WhiteNonHispanicOriginations,BlackOriginations,LatinxOriginations,AAPIOriginations,Applications,Denials,Fallout
0,55079000101,29,7,10,4,3,52,6,10
1,55079000102,40,9,22,1,4,67,7,7
2,55079000201,36,13,13,4,3,62,8,7
3,55079000202,81,28,37,9,5,136,13,19
4,55079000301,34,21,7,1,3,52,5,5


In [154]:
tract_summary['OriginationRate'] = round(tract_summary['TotalOriginations'] / tract_summary['Applications'], 6)
tract_summary['DenialRate'] = round(tract_summary['Denials'] / tract_summary['Applications'], 6)
tract_summary['FalloutRate'] = round(tract_summary['Fallout'] / tract_summary['Applications'], 6)

In [155]:
tract_summary.sort_values(by='census_tract', ascending=True).head(20)

,census_tract,TotalOriginations,WhiteNonHispanicOriginations,BlackOriginations,LatinxOriginations,AAPIOriginations,Applications,Denials,Fallout,OriginationRate,DenialRate,FalloutRate
0,55079000101,29,7,10,4,3,52,6,10,0.557692,0.115385,0.192308
1,55079000102,40,9,22,1,4,67,7,7,0.597015,0.104478,0.104478
2,55079000201,36,13,13,4,3,62,8,7,0.580645,0.129032,0.112903
3,55079000202,81,28,37,9,5,136,13,19,0.595588,0.095588,0.139706
4,55079000301,34,21,7,1,3,52,5,5,0.653846,0.096154,0.096154
5,55079000302,17,7,7,3,1,23,1,1,0.739130,0.043478,0.043478
6,55079000303,18,8,3,2,4,33,3,7,0.545455,0.090909,0.212121
7,55079000304,49,9,24,5,9,79,6,9,0.620253,0.075949,0.113924
8,55079000400,23,13,4,3,3,39,5,4,0.589744,0.128205,0.102564
9,55079000501,36,15,14,4,4,67,8,7,0.537313,0.119403,0.104478


In [147]:
target_df.sort_values(by='census_tract', ascending=True).head(20)

,census_tract,TotalOriginations,WhiteNonHispanicOriginations,BlackOriginations,LatinxOriginations,AAPIOriginations,Applications,Denials,Fallout,OriginationRate,DenialRate,FalloutRate
0,55079000101,29,3,10,2,3,52,6,11,0.557692,0.115385,0.211538
177,55079000102,40,6,22,1,4,67,7,11,0.597015,0.104478,0.164179
45,55079000201,36,10,12,4,3,62,8,8,0.580645,0.129032,0.129032
25,55079000202,81,21,37,8,5,133,13,25,0.609023,0.097744,0.187970
46,55079000301,34,18,7,1,3,53,5,6,0.641509,0.094340,0.113208
47,55079000302,17,5,7,2,1,23,1,1,0.739130,0.043478,0.043478
48,55079000303,18,6,3,2,4,33,3,8,0.545455,0.090909,0.242424
21,55079000304,49,6,24,5,9,79,6,11,0.620253,0.075949,0.139241
28,55079000400,23,11,4,3,3,40,5,4,0.575000,0.125000,0.100000
29,55079000501,36,12,13,4,4,68,8,10,0.529412,0.117647,0.147059
